# Computating statistics of Benchmark Data

This notebook serves for the following steps within the Benchmark:
* It gathers all the information recorded and checks if the data is stable enough (this means if the Std of some measurements is small enough)
* If this is not the case the user gets toled to record another bag
* If this is the case the data is analysed and the means are calculated
* 1 final .yaml file called `BAGNAME_benchmark_final_results.yaml` is designed including on one hand all the engineering data and on the other hand the overall results of the actual behaviour. This file is stored into the folder `~/behaviour-benchmarking/data/BenchmarkXY/benchmarks/final`. Please change the variable Benchmark to the name of your Benchmark (or the name of the foldet containing all information and where you wan to store the results

Please run cell after cell reading carefully the instructions between the cell as well as the outputs computed.


In [ ]:
Benchmark = 'BenchmarkXY'

In [ ]:
#!/usr/bin/env python
# coding: utf-8

#All imports necessary for this Notebook
import contracts
contracts.disable_all()

import geometry as geo
import math 
import numpy as np
from os import path, listdir
from scipy import stats
import yaml
import json
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
from math import ceil
from scipy import interpolate
from os import path, listdir
from ipywidgets import FileUpload
import statistics
import collections

import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_svg, ipython_draw_html
from duckietown_world.world_duckietown.tile import get_lane_poses
from duckietown_world import draw_static

Below there are some functions that are used later

In [ ]:
def relative_pose(q0, q1):
    "Computes the relative pose between two points in SE2"
    return geo.SE2.multiply(geo.SE2.inverse(q0), q1)

class AFakeBar(dw.PlacedObject):
    "Ellipse object with a large ration between the radii"

    def __init__(self, len=0, fill_opacity=0.5, color='pink', *args, **kwargs):
        self.len = len
        self.fill_opacity = fill_opacity
        self.color = color
        dw.PlacedObject.__init__(self, *args, **kwargs)

    def draw_svg(self, drawing, g):
        # drawing is done using the library svgwrite
        c = drawing.ellipse(center=(0, 0), r=(0.03,self.len), fill=self.color, fill_opacity=self.fill_opacity)
        g.add(c)
        # draws x,y axes
        dw.draw_axes(drawing, g)
        
def find_nearest_2d(mid_line, point, theta):
    """Function to find the nearest point on the midle line to a specific point in 2d"""
    """It then calculates the relative x and y offset of the point to the nearest point on the center line"""
    """ as well as the relative angle of the April Tag on your Duckiebot compared to the cener line"""
#     print(value)
    min_dist = 100000
    rel_offset_cr_min = 10000
#     print(type(mid_line))
    start = True
    indx = 0
    for i in range(1, len(mid_line)):
        xs_c = mid_line[i].p[0]
        ys_c = mid_line[i].p[1]
        xs_p = mid_line[i-1].p[0]
        ys_p = mid_line[i-1].p[1]
        p1 = np.array([xs_p,ys_p])
        p2 = np.array([xs_c,ys_c])
        p3 = np.array([point[0],point[1]])
        rel_offset_cr = np.cross(p2-p1,p3-p1)/np.linalg.norm(p2-p1)
        if rel_offset_cr < rel_offset_cr_min:
            rel_offset_cr_min = rel_offset_cr
            indx = i
        dist = (point[0]-xs_c)**2 + (point[1]-ys_c)**2
        if dist < min_dist:
            min_dist = dist
            
        
            
    rel_x = point[0] - mid_line[indx].p[0] 
    rel_y = point[1] - mid_line[indx].p[1] 
    rel_angle = mid_line[indx].theta
    theta_rel = np.arctan2(np.mean(np.sin(theta-rel_angle)),np.mean(np.cos(theta-rel_angle)))
    
#     indx = (mid_line.index(idx))    
    return indx, rel_x, rel_y, theta_rel, rel_offset_cr_min

class Circle(dw.PlacedObject):
    "Circle object."

    def __init__(self, radius, color='pink', *args, **kwargs):
        self.radius = radius
        self.color = color
        dw.PlacedObject.__init__(self, *args, **kwargs)

    def draw_svg(self, drawing, g):
        # drawing is done using the library svgwrite
        c = drawing.circle(center=(0, 0), r=self.radius, fill=self.color)
        g.add(c)
        # draws x,y axes
        dw.draw_axes(drawing, g)

    def extent_points(self):
        # set of points describing the boundary
        L = self.radius
        return [(-L, -L), (+L, +L)]
    
    
def interpolate_custom(q0, q1, alpha):
    "Interpolates between two points in SE2, given a coefficient alpha."
    q1_from_q0 = relative_pose(q0, q1)
    vel = geo.SE2.algebra_from_group(q1_from_q0)
    rel = geo.SE2.group_from_algebra(vel * alpha)
    q = geo.SE2.multiply(q0, rel)
    return q

def get_global_center_line(map, used_lane_segs, global_segs_SE2, pts_per_segment):
    "Builds a center line for all the used lanes in the global coordinate frame."
    center_line = []
    center_line_global = []
    center_line_global_tfs = []
    
    # The number of points genereated for the center line depends on the tile 
    # mid is the number of points for a straight tile
    # long is the number of points for a left curve tile
    # short is the number of points for a right curve tile
    for i, lane_segment in enumerate(used_lane_segs):
        if lane_segment[2] == 'straight':
            n_inter = int(pts_per_segment['mid'])
        elif lane_segment[-1] == 'lane2':
            n_inter = int(pts_per_segment['long'])
        elif lane_segment[-1] == 'lane1':
            n_inter = int(pts_per_segment['short'])
        lane = map[lane_segment]

        # The end point is part of next tile
        steps = np.linspace(0, len(lane.control_points) - 1, num=n_inter, endpoint=False)

        for beta in steps:
            center_point_local_SE2 = lane.center_point(beta)
            center_line.append(center_point_local_SE2)

            # get SE2 of the point in global coords
            center_point_global_SE2 = geo.SE2.multiply(global_segs_SE2[lane_segment],
                                                       center_point_local_SE2)

            center_line_global.append(center_point_global_SE2)
            center_line_global_tfs.append(dw.SE2Transform.from_SE2(center_point_global_SE2))

    
    
    return center_line_global, center_line_global_tfs

def get_used_lanes_mine(trajectories):
    """Returns a list with all used lanes and a dictionary containing the transform to each lane segment."""
    """It also calculates the number of completed laps, the time needed per tile and it counts the number"""
    """of tiles covered (total as well as specific for different types)"""
    """Moreover it checks if the Duckiebot had a crash or drives too slow -> if the center of the April Tag"""
    """of the  Duckiebot takes more than 30 seconds to get across one tile the Benchmark is stoped there"""
    """The time when this happened is saved and the trajectories are shorten to that time"""
    
    # If in future for another Benchmark there are other tiles part of the loop just add a dictionary for them as well
    used_lane_segs = set()
    used_lane_segs_list = []
    lane_segs_tfs = dict()
    last_lane_seg = dict()
    prev_lane_seg = ()
    current_lane_seg = ()
    start_tile = ()
    
    total_nb_of_tiles = 0
    nb_straight_tiles = 0
    nb_curve_left = 0
    nb_curve_right = 0
    nb_complete_laps = 0
    
    too_slow = False
    
    first_time_on_tile = 0.0
    start = False
    new_tile = False
    count = 0
    
    
    for traj in trajectories:
        for pose in traj:
            count += 1
            try:
                tl = list(get_lane_poses(m, pose))[0]
                lane_segment_name = tl.lane_segment_fqn
                if not start:
                    # if other tiles are part of the loop, just add another if condition with the name of the tile
                    total_nb_of_tiles += 1
                    # checks what kind of tile that it is
                    if lane_segment_name[2] == "straight":  
                        nb_straight_tiles += 1  
                    elif lane_segment_name[2] == "curve_left": 
                        nb_curve_left += 1;
                    elif lane_segment_name[2] == "curve_right":
                        nb_curve_right += 1
                    
                    start_tile = lane_segment_name
                    current_lane_seg = lane_segment_name
                    prev_lane_seg = lane_segment_name
                    start = True
                    
                if lane_segment_name[1] == current_lane_seg[1]:
                    new_tile = False
                    # the following condoition checks if the Duckiebot drives too slow or not

                elif lane_segment_name[1] != current_lane_seg[1]:
                    new_tile = True
                    # if other tiles are part of the loop, just add another if condition with the name of the tile
                    total_nb_of_tiles += 1
                    # checks what kind of tile that it is
                    if lane_segment_name[2] == "straight":  
                        nb_straight_tiles += 1  
                    elif lane_segment_name[2] == "curve_left": 
                        nb_curve_left += 1;
                    elif lane_segment_name[2] == "curve_right":
                        nb_curve_right += 1
                        
                    current_lane_seg = lane_segment_name
                    
                    if lane_segment_name[1] == start_tile[1]:
                        print("new round")
                        nb_complete_laps +=1
                
                #checks if the lane segment appears for the first time or not
                #if it appears for the first time the new lane segment is added to the list of used lane segments
                if lane_segment_name not in used_lane_segs:
                    used_lane_segs.add(lane_segment_name)
                    used_lane_segs_list.append(lane_segment_name)
                    lane_segs_tfs[lane_segment_name] = tl.lane_segment_transform.asmatrix2d().m
            except IndexError:
                pass

    return used_lane_segs_list, lane_segs_tfs, nb_complete_laps, total_nb_of_tiles, nb_straight_tiles, \
nb_curve_left, nb_curve_right

def get_used_lanes(trajectories):
    """Returns a list with all used lanes and a dictionary containing the transform to each lane segment."""
    used_lane_segs = set()
    used_lane_segs_list = []
    lane_segs_tfs = dict()

    for traj in trajectories:
        for pose in traj:
            try:
                tl = list(get_lane_poses(m, pose))[0]
                lane_segment_name = tl.lane_segment_fqn

                if lane_segment_name not in used_lane_segs:
                    used_lane_segs.add(lane_segment_name)
                    used_lane_segs_list.append(lane_segment_name)
                    lane_segs_tfs[lane_segment_name] = tl.lane_segment_transform.asmatrix2d().m
            except IndexError:
                pass
    return used_lane_segs_list, lane_segs_tfs

def get_interpolated_points(center_line, trajectories):
    """Generates an interpolated point for each point on the center line, for each trajectory as long as the point
    lies between two trajectory points."""
    closest_behind = [None] * len(trajectories)
    interpolated_trajectories = []
    for center_point in center_line:
        interpolated_points = []
        for idx_t, traj in enumerate(trajectories):
            interpolated_point_traj = None
            begin_t = closest_behind[idx_t] if closest_behind[idx_t] else 0
            for idx_point in range(begin_t, len(traj)):
                if a_behind_b(a=traj[idx_point], b=center_point):
                    closest_behind[idx_t] = idx_point
                    continue

                if closest_behind[idx_t] is None:
                    # If there is no point behind we cannot compute the interpolation
                    interpolated_point_traj = None
                    break
                else:
                    try:
                        interpolated_point_traj = interpolate_magic(center_point,
                                                                    traj[closest_behind[idx_t]],
                                                                    traj[closest_behind[idx_t] + 1])
                        break

                    except IndexError:
                        print('The index is outside the list!')
                        interpolated_point_traj = None
                        break
            interpolated_points.append(interpolated_point_traj)
        interpolated_trajectories.append(interpolated_points)
    return interpolated_trajectories


def a_behind_b(a=None, b=None):
    """Check if a is behind b wrt the heading direction of a."""
    if a is None or b is None:
        return False
    rel_pose = relative_pose(b, a)
    return dw.SE2Transform.from_SE2(rel_pose).p[0] < 0


def interpolate_magic(center_pt, previous_pt, next_pt):
    """Returns an interpolated point between previoust_pt and next_pt at the height of center_pt"""
    tf_prev = relative_pose(center_pt, previous_pt)
    d_prev = dw.SE2Transform.from_SE2(tf_prev).p[0]

    tf_next = relative_pose(center_pt, next_pt)
    d_next = dw.SE2Transform.from_SE2(tf_next).p[0]

    alpha = np.abs(d_prev) / (np.abs(d_prev) + d_next)
    interpolated_pt = interpolate_custom(previous_pt, next_pt, alpha)
    return interpolated_pt


def get_trajectories_statistics(trajectories,center_line):
    """Computes mean trajectory and std deviations for y and angle given a list of trajectories sampled at the same x"""
    mean_tfs = []
    std_y = []
    mean_offset = []
    cv_y = []
    cv_heading = []
    std_heading = []
    mean_heading = []

    start_idx = None
    end_idx = None
    # We need to find the first amd last index for which all trajectories have a point
    for idx, trajs_points in enumerate(trajectories):
        if all(trajs_points) and start_idx is None:
            start_idx = idx
        elif not all(trajs_points) and start_idx is not None:
            end_idx = idx
            break
    end_idx = -1 if end_idx is None else end_idx
    complete_trajectories = trajectories[start_idx:end_idx]
    
    for tfs in complete_trajectories:
        xs = [tf.p[0] for tf in tfs]
        ys = [tf.p[1] for tf in tfs]
        headings = [tf.theta for tf in tfs]
        mean_x = np.mean(xs)
        mean_y = np.mean(ys)
        point = [mean_x , mean_y]
        # To compute mean angles we need to pay attention
        mean_angle = np.arctan2(np.mean(np.sin(headings)),np.mean(np.cos(headings)))
        mean_tfs.append(dw.SE2Transform.from_SE2(geo.SE2_from_translation_angle([mean_x, mean_y], mean_angle)))
        
        cur_ind = []
        cur_offset_mine = []
        cur_heading_mine = []
        
        #find closest point on center_line to the mean of all of the points at this specific x
        indx, x_rel, y_rel, theta_rel, rel_offset_cr_min = find_nearest_2d(center_line,point, mean_angle)
        for i in range(0, len(xs)):
            #transform the point to SE2
            point_cur_tf = geo.SE2_from_translation_angle([xs[i], ys[i]], headings[i])
            #for each point get relative position to the center line
            relative_tf_mine = dw.SE2Transform.from_SE2(relative_pose(point_cur_tf, center_line[indx].as_SE2()))
            #extract the offset and the heading angle deviation of the point relative to the center line
            cur_offset_mine.append(relative_tf_mine.p[1].item())
            cur_heading_mine.append(relative_tf_mine.theta)


        # Compute all transforms wrt to the mean trajectory to compute the standard deviations
        #lateral_deviation = [(mean_x-t.p[0])*np.sin(t.theta)+(mean_y-t.p[1])*np.cos(t.theta) for t in tfs]
        lateral_deviation = []
        mean_point = geo.SE2_from_translation_angle([mean_x, mean_y], mean_angle)
        for t in tfs:
            relative_tf = dw.SE2Transform.from_SE2(relative_pose(mean_point, t.as_SE2()))
            lateral_deviation.append(relative_tf.p[1])
        
        relative_tf_mine = dw.SE2Transform.from_SE2(relative_pose(mean_point, center_line[indx].as_SE2()))
        
        offset_wt_interp_all.append(relative_tf_mine.p[1].item())
        angle_wt_interp_all.append(relative_tf_mine.theta)

        #calculate the avg and std lateral offset of the points (that are at a specific x position) of all the  
        #different trajectories to the center line
        std_y_cur = float(np.round(np.std(cur_offset_mine),6))
        mean_y_cur = float(np.round(np.mean(cur_offset_mine),6))

        mean_offset.append(float(np.round(np.mean(cur_offset_mine),6)))
        std_y.append(float(np.round(np.std(cur_offset_mine),6)))
        if mean_y_cur != 0.0:
            cv_y.append(float(np.round(abs(std_y_cur/mean_y_cur),6)))
        else:
            if std_y_cur == 0.0:
                cv_y.append(0.0)
            else:
                cv_y.append(10.0)
                
        #calculate the avg and std heading deviation of the points (that are at a specific x position) of all the  
        #different trajectories to the center line
        std_angle_cur = float(np.round(stats.circstd(cur_heading_mine, low=-math.pi, high=math.pi),6))
        mean_angle_cur = float(np.round(stats.circmean(cur_heading_mine, low=-math.pi, high=math.pi),6))
        mean_heading.append(mean_angle_cur)
        std_heading.append(float(np.round(stats.circstd(cur_heading_mine, low=-math.pi, high=math.pi),6)))
        if mean_angle_cur != 0.0:
            cv_heading.append(float(np.round(abs(std_angle_cur/mean_angle_cur),6)))
        else:
            if std_angle_cur == 0.0:
                cv_heading.append(0.0)
            else:
                cv_heading.append(10.0)
        
    return mean_tfs, std_y, std_heading, start_idx, end_idx,cv_y, cv_heading, mean_offset, mean_heading

def get_trajectories_statistics_mean_traj(trajectories, center_line):
    """For each point on the trajectory of the Duckiebot, the relative offset as well as its angle of the center of """
    """the April Tag of your Duckiebot is calculated"""
    mean_tfs = []
    std_y = []
    std_heading = []

    complete_trajectories = trajectories[:]
    lateral_deviation_tes = []
    rel_offset_cr = []
    theta_rel_cr = []
    
    for tfs in complete_trajectories:
        xs = tfs.p[0]
        ys = tfs.p[1]      
        headings = tfs.theta
        mean_x = np.mean(xs)
        mean_y = np.mean(ys)
        point = [mean_x , mean_y]
        mean_angle = np.arctan2(np.mean(np.sin(headings)),np.mean(np.cos(headings)))
        
        mean_point = geo.SE2_from_translation_angle([mean_x, mean_y], mean_angle)
        
        
        indx, x_rel, y_rel, theta_rel, rel_offset_cr_min = find_nearest_2d(center_line,point, mean_angle)
        
        relative_tf = dw.SE2Transform.from_SE2(relative_pose(mean_point, center_line[indx].as_SE2()))
        
        rel_offset_cr.append(rel_offset_cr_min)
        theta_rel_cr.append(theta_rel)
        
        # Compute all transforms wrt to the mean trajectory to compute the standard deviations
        #lateral_deviation = [(mean_x-t.p[0])*np.sin(t.theta)+(mean_y-t.p[1])*np.cos(t.theta) for t in tfs]
        lateral_deviation_tes.append((x_rel)*np.sin(theta_rel)+(y_rel)*np.cos(theta_rel))
#         print((x_rel)*np.sin(theta_rel)+(y_rel)*np.cos(theta_rel))
        offset_wt_non_interp_all.append(relative_tf.p[1].item())
        angle_wt_non_interp_all.append(relative_tf.theta)
        
    return lateral_deviation_tes

def Average(lst): 
    """Calculates the average of a list"""
    lst_abs = [abs(x) for x in lst]
    return sum(lst_abs) / len(lst) 

def get_unit(info):
    """Function returning the unit of the different informations""" 
    if info == 'Number_of_completed_laps':
        unit = 'laps'
    elif info == 'Number_of_tiles_covered':
        unit = 'tiles'
    elif info == 'Avg_time_needed_per_tile':
        unit = 'seconds per tile'
    elif info == 'Time_needed_per_straight_tile_sec':
        unit = 'seconds per tile'
    elif info == 'Time_needed_per_curved_tile':
        unit = 'seconds per tile'
    elif info == 'Length_of_recorded_bag':
        unit = 'seconds' 
    elif info == 'Actual_length_of_benchmark':
        unit = 'seconds'   
    elif info == 'Theoretical_length_of_benchmark':
        unit = 'seconds'
    elif info == 'Out_of_sight':
        unit = ' '  
    elif info == 'Tolerance_out_of_sight':
        unit = 'seconds' 
    elif info == 'Time_out_of_sight':
        unit = 'seconds' 
    elif info == 'Too_slow':
        unit = ' '  
    elif info == 'Time_too_slow':
        unit = 'seconds' 
    elif info == 'Tolerance_too_slow_sec':
        unit = 'seconds' 
    elif info == 'Position_too_slow':
        unit = ' '
    elif info == 'Abs_Ground_truth_wt_std_offset_non_interp':
        unit = 'meters' 
    elif info == 'Abs_Ground_truth_wt_std_angle_non_interp':
        unit = 'degree' 
    elif info == 'Abs_Ground_truth_wt_mean_offset_non_interp':
        unit = 'meters' 
    elif info == 'Abs_Ground_truth_wt_mean_angle_non_interp':
        unit = 'degree' 
    elif info == 'Abs_Ground_truth_wt_median_offset_non_interp':
        unit = 'meters' 
    elif info == 'Abs_Ground_truth_wt_median_angle_non_interp':
        unit = 'degree' 
    elif info == 'Abs_Ground_truth_wt_std_offset_interp':
        unit = 'meters' 
    elif info == 'Abs_Ground_truth_wt_std_angle_interp':
        unit = 'degree' 
    elif info == 'Abs_Ground_truth_wt_mean_offset_interp':
        unit = 'meters' 
    elif info == 'Abs_Ground_truth_wt_mean_angle_interp':
        unit = 'degree' 
    elif info == 'Abs_Ground_truth_wt_median_offset_interp':
        unit = 'meters' 
    elif info == 'Abs_Ground_truth_wt_median_angle_interp':
        unit = 'degree'
    elif info == 'Abs_Measurements_db_std_offset':
        unit = 'meters'
    elif info == 'Abs_Measurements_db_std_angle':
        unit = 'degree' 
    elif info == 'Abs_Measurements_db_mean_offset':
        unit = 'meters' 
    elif info == 'Abs_Measurements_db_mean_angle':
        unit = 'degree' 
    elif info == 'Abs_Measurements_db_median_offset':
        unit = 'meters' 
    elif info == 'Abs_Measurements_db_median_angle':
        unit = 'degree' 
    elif info == 'std_diff_btw_estimation_and_ground_truth_offset':
        unit = 'meters' 
    elif info == 'std_diff_btw_estimation_and_ground_truth_angle':
        unit = 'degree'
    elif info == 'mean_diff_btw_estimation_and_ground_truth_offset':
        unit = 'meters' 
    elif info == 'mean_diff_btw_estimation_and_ground_truth_angle':
        unit = 'degree'
    elif info == 'median_diff_btw_estimation_and_ground_truth_offset':
        unit = 'meters' 
    elif info == 'median_diff_btw_estimation_and_ground_truth_angle':
        unit = 'degree'
    elif info == 'offset_db':
        unit = 'meters' 
    elif info == 'angle_db':
        unit = 'degree'
    elif info == 'offset_wt_non_interp':
        unit = 'meters' 
    elif info == 'angle_wt_non_interp':
        unit = 'degree'
    else:
        unit = 'Uuuups' 
    
    return unit

def count_true(results):
    """Function that calculates how many times the boolean True is within the list results"""
    counter = 0
    for i in range(0,len(results)):
        if results[i] == True:
            counter += 1
    return counter


def save_data():
    """Function that saves the results in a the corresponding yaml files"""
    # Path and name of the created yaml file, the file 'BAGNAME_eng_perf_data_all.yaml' can be found
    # in the folder behaviour_benchmarking/data/BenchmarkXY/out
    eng_perf_data_all = path.join(outdir, name + '_eng_perf_data_all.yaml')

    # Safe the eng_data_all dictionary in a yaml file called eng_perf_data_all.yaml    
    with open(eng_perf_data_all, 'w') as yaml_file:
        yaml.dump(eng_data_all, yaml_file, default_flow_style=False)   

    # Path and name under which the static_things dictionary is saved
    static_things_path = path.join(outdir, name + '_software_information.yaml')

    # Path and name of the created yaml file, the file 'BAGNAME_software_information.yaml' can be found
    # in the folder behaviour_benchmarking/data/BenchmarkXY/out
    with open(static_things_path, 'w') as yaml_file:
        yaml.dump(static_things, yaml_file, default_flow_style=False)
        
    final_results.update({'Engineering Data': {'Performance':eng_data_all,'Static':static_things}})
    
    with open(benchmark_final_results, 'w') as yaml_file:
        yaml.dump(final_results, yaml_file, default_flow_style=False)
        

## Standard deviation calculation of trajectories

First of all, the overall mean trajectories measured by the Watchtowers (Ground Truth are compared) and the Standard Deviation is calculated. This means, that for each position within the loop, the average position (offset and heading) out of all the experiments ran of the Duckiebot measured by the Watchtowers (Ground Truth) is calculated. Based on this the standard deviations of the offset and the heading are also computed for each position within the loop.  

If any of the standard deviation is too high the user is asked to run another experiment and collect more data.
This avoids a lucky punch of the performance.

To be able to calculate all this, we need to extract the ground truth trajectory (measured by the Watchtowers) out of each `BAGNAME_benchmark_results_test_XY.yaml` file.

But first of all lets load all the `BAGNAME_benchmark_results_test_XY.yaml` available and extract all the data and check if the experiments have actually been done for the same type of Benchmark


Please adapt `logs_path` if you stored your `BAGNAME_benchmark_results_test_XY.yaml` in another folder.
Also, please note that it is necessary to have at least 2 different yaml files in the folder. Please make sure that the bags are named considering the naming convention explained in the documents. This means that within the folder there should at least be the files: `BAGNAME_benchmark_results_test_01.yaml` and `BAGNAME_benchmark_results_test_02.yaml`)


In [ ]:
experiment_dir = ''
logs_path = path.join(experiment_dir, '../data/'+Benchmark+'/benchmarks/same_bm')
logs_path_save = path.join(experiment_dir, '../data/'+ Benchmark)

#Looking what files that are fined within the destination logs_path
localization_logs = [path.join(logs_path, f) for f in listdir(logs_path) if path.isfile(path.join(logs_path, f))]
print(f'Logs found: {localization_logs}')

#Calculates the number of Benchmarks found
nb_bm_found = len(localization_logs)
all_logs = []
names=[]

# loads all the data found in all the different yaml files and stores it into all_logs
i = 0
for filename in localization_logs:
    with open(filename, 'r') as file:
        current = []
        names.append(path.basename(filename))
        current.append(yaml.safe_load(file))
        
    all_logs.append(current)

    i += 1

#Number of experiments ran so far
number_of_tests = i

Please check which `BAGNAME_benchmark_results_test_XY.yaml` is listed first above in the Logs found.
And change the variable `first_loaded_bag` to `_benchmark_results_test_XY.yaml`

In [ ]:
first_loaded_bag = "_benchmark_results_test_XY.yaml"
#extract the BAGNAME
name_a = path.basename(names[0])
name = name_a.replace(first_loaded_bag,"")

Please change the Map_Name below if you ran the experiments for your Benchmark on a different map than linus_loop

In [ ]:
Map_Name = 'linus_loop'

Below some lists are created to sort the different results that can be found within the files can be seperated from each other

In [ ]:
#list of results that have boolean as entries
no_meaningful_rel_comp = ['Out_of_sight', 'Too_slow']
#list of results that are not defined yet
list_of_comp_todo = ['Position_too_slow', 'Time_needed_per_straight_tile_sec', 'Time_needed_per_curved_tile']
#list of thing we want to check for low enough std
meaningful_comp_results = ['Number_of_tiles_covered','Avg_time_needed_per_tile','mean_diff_btw_estimation_and_ground_truth_offset',\
                          'mean_diff_btw_estimation_and_ground_truth_angle', 'Abs_Measurements_db_mean_offset',\
                          'Abs_Measurements_db_mean_angle','Abs_Ground_truth_wt_mean_offset_interp',\
                          'Abs_Ground_truth_wt_mean_angle_interp','Abs_Ground_truth_wt_mean_offset_non_interp',\
                          'Abs_Ground_truth_wt_mean_angle_non_interp']
#list of all the constant tolerances 
tolerances = ['Tolerance_out_of_sight', 'Tolerance_too_slow_sec', 'Theoretical_length_of_benchmark']
#list including all resultrs that just stored trajectories
raw_traj_info = ['int_trajs', 'all_trajectories', 'time_wt', 'time_db', 'time_db_true','angle_wt_interp','angle_db',\
                      'angle_wt_non_interp','angle_db_true','offset_wt_interp','offset_db','offset_wt_non_interp',\
                      'offset_db_true', 'all_trajectories_db']
#list including all resultrs that just stored trajectories information
overall_traj_info = ['angle_wt_interp','angle_db', 'angle_wt_non_interp','angle_db_true','offset_wt_interp',\
                     'offset_db','offset_wt_non_interp', 'offset_db_true']
#list with all results of which we want to calculate mean etc but don't care if std is low enough
general_info = ['Length_of_recorded_bag','Actual_length_of_benchmark','Time_out_of_sight','Time_too_slow',\
               'Abs_Ground_truth_wt_median_offset_non_interp','Abs_Ground_truth_wt_median_angle_non_interp',\
               'Abs_Ground_truth_wt_std_offset_non_interp','Abs_Ground_truth_wt_std_angle_non_interp',\
               'Abs_Ground_truth_wt_median_offset_interp','Abs_Ground_truth_wt_median_angle_interp',\
               'Abs_Ground_truth_wt_std_offset_interp','Abs_Ground_truth_wt_std_angle_interp',\
               'Abs_Measurements_db_std_offset','Abs_Measurements_db_std_angle','Abs_Measurements_db_median_offset',\
               'Abs_Measurements_db_median_angle','std_diff_btw_estimation_and_ground_truth_offset',\
               'std_diff_btw_estimation_and_ground_truth_angle','median_diff_btw_estimation_and_ground_truth_offset',\
               'median_diff_btw_estimation_and_ground_truth_angle','']

#Calculates the number of properties found per file
nb_of_properties = len(all_logs[0][0]['Results'])



The cell below checks if the Benchmarks for which the mean respectively standard deviation are calculated are of the same type. 
If not, such a comparison does not make any sense.

In [ ]:
for i in range(0, nb_bm_found):
    if all_logs[0][0]['Benchmark_Type'] != all_logs[i][0]['Benchmark_Type']:
        print("This avg calculation work as the results come from two different Benchmarks, please stop here\
        and upload results from the same Benchmark type")

In [ ]:
# some paths to outputs that will be generated
benchmark_std = path.join(experiment_dir, 'out/benchmark_std.yaml')
benchmark_mean = path.join(experiment_dir, 'out/'+ name + '_benchmark_mean.yaml')
benchmark_final_results = path.join(experiment_dir, '../data/'+Benchmark+'/benchmarks/final/'+ name + '_benchmark_final_results.yaml')
benchmark_final_results_wo_eng_data = path.join(experiment_dir, 'out/'+ name + '_benchmark_final_results_wo_eng_data.yaml')
benchmark_std_graph = path.join(experiment_dir, 'out/benchmark_std_graph.jpg')
benchmark_boxplot_graph = path.join(experiment_dir, 'out/benchmark_boxplot_graph.jpg')

#some lists needed later
offset_wt_non_interp_all = []
angle_wt_non_interp_all = []
offset_wt_interp_all = []
angle_wt_interp_all = []

The cell below extracts all the needed information of all the different results file. The names of the lists where they are stored in are self explenatory.

In [ ]:
# All the trajectories measured by the Watchtowers (ground truth) over all experiments
all_trajectories = []
for log in all_logs:
    all_cur_traj = []
    for i in range(0, len(log[0]['Results']['all_trajectories'])):
        cur_traj = np.array(log[0]['Results']['all_trajectories'][i])
        all_cur_traj.append(cur_traj)
    all_trajectories.append(all_cur_traj)

# All the relative pose estimations (offset and heading) done by the Duckiebot over the time of the Benchmark
all_trajectories_db = []
for log in all_logs:
    all_cur_traj = []
    for i in range(0, len(log[0]['Results']['all_trajectories_db'])):
        cur_traj = np.array(log[0]['Results']['all_trajectories_db'][i])
        all_cur_traj.append(cur_traj)
    all_trajectories_db.append(all_cur_traj)

# Timestamps of the Watchtower measurements
time_wt = {}
for k, log in enumerate(all_logs):
    all_cur_time = []
    for i in range(0, len(log[0]['Results']['time_wt'])):
        all_cur_time.append(log[0]['Results']['time_wt'][i])
    time_wt.update({k: all_cur_time})

# Timestamps of the Duckiebote pose estimations
time_db = {}
for k, log in enumerate(all_logs):
    all_cur_time = []
    for i in range(0, len(log[0]['Results']['time_db'])):
        all_cur_time.append(log[0]['Results']['time_db'][i])
    time_db.update({k: all_cur_time})    

# Timestamps of the Duckiebot pose estimations recorded from on the Duckiebot itself (if available)
time_db_true = {}
for k, log in enumerate(all_logs):
    all_cur_time = []
    for i in range(0, len(log[0]['Results']['time_db_true'])):
        all_cur_time.append(log[0]['Results']['time_db_true'][i])
    time_db_true.update({k: all_cur_time})

# Offsets estimated by the Duckiebot
offset_db = {}
for k, log in enumerate(all_logs):
    all_cur_offset = []
    for i in range(0, len(log[0]['Results']['offset_db'])):
        all_cur_offset.append(log[0]['Results']['offset_db'][i])
    offset_db.update({k: all_cur_offset})
    
# Offsets calculated by the Duckiebot, recorded on the Duckiebot itself (if available)
offset_db_true = {}
for k, log in enumerate(all_logs):
    all_cur_offset = []
    for i in range(0, len(log[0]['Results']['offset_db_true'])):
        all_cur_offset.append(log[0]['Results']['offset_db_true'][i])
    offset_db_true.update({k: all_cur_offset})

# Offsets calculated by from the interpolated Watchtower measurements(ground truth)
offset_wt_interp = {}
for k, log in enumerate(all_logs):
    all_cur_offset = []
    for i in range(0, len(log[0]['Results']['offset_wt_interp'])):
        all_cur_offset.append(log[0]['Results']['offset_wt_interp'][i])
    offset_wt_interp.update({k: all_cur_offset})

# Offsets calculated by from the non-interpolated Watchtower measurements(ground truth)    
offset_wt_non_interp = {}
for k, log in enumerate(all_logs):
    all_cur_offset = []
    for i in range(0, len(log[0]['Results']['offset_wt_non_interp'])):
        all_cur_offset.append(log[0]['Results']['offset_wt_non_interp'][i])
    offset_wt_non_interp.update({k: all_cur_offset})

# Heading estimated by the Duckiebot
angle_db = {}
for k, log in enumerate(all_logs):
    all_cur_angle = []
    for i in range(0, len(log[0]['Results']['angle_db'])):
        all_cur_angle.append(log[0]['Results']['angle_db'][i])
    angle_db.update({k: all_cur_angle})

# Heading calculated by the Duckiebot, recorded on the Duckiebot itself (if available)
angle_db_true = {}
for k, log in enumerate(all_logs):
    all_cur_angle = []
    for i in range(0, len(log[0]['Results']['angle_db_true'])):
        all_cur_angle.append(log[0]['Results']['angle_db_true'][i])
    angle_db_true.update({k: all_cur_angle})
    
# Heading calculated by from the interpolated Watchtower measurements(ground truth)
angle_wt_interp = {}
for k, log in enumerate(all_logs):
    all_cur_angle = []
    for i in range(0, len(log[0]['Results']['angle_wt_interp'])):
        all_cur_angle.append(log[0]['Results']['angle_wt_interp'][i])
    angle_wt_interp.update({k: all_cur_angle})

# Heading calculated by from the non-interpolated Watchtower measurements(ground truth)
angle_wt_non_interp = {}
for k, log in enumerate(all_logs):
    all_cur_angle = []
    for i in range(0, len(log[0]['Results']['angle_wt_non_interp'])):
        all_cur_angle.append(log[0]['Results']['angle_wt_non_interp'][i])
    angle_wt_non_interp.update({k: all_cur_angle})

In [ ]:
# saves the runtimes of the different bags recorded and also finds the minimum
runtime = []
for i in range (0, nb_bm_found):
    runtime.append(time_wt[i][-1])
    
min_runtime = float(min(runtime))

In [ ]:
# Load the map used for these Benchmarks
try:
    del m    
except:
    pass
m = dw.load_map(Map_Name)

# Calculate the used lane segments by all the trajectories
used_lane_segments_list, lane_segments_SE2 = get_used_lanes(all_trajectories)

# Calculates the length of the trajectories
mid = 30 
cnt = collections.Counter()
for x in used_lane_segments_list:
    cnt[x[2]] +=1

# Number of interpolation points of each tile (approximation, need to do it properly)
pts_per_segment = {
    'short': int(mid*1/8*math.pi),
    'mid': (mid),
    'long': int(mid*3/8*math.pi),
}

# Compute the center line that we will use to resample
center_line_global, center_line_global_tfs = get_global_center_line(m,
                                                                    used_lane_segments_list,
                                                                    lane_segments_SE2,
                                                                    pts_per_segment)



In [ ]:
# Base transform if the plotting map is not the same as the evaluation map (i.e. plotting a subset 
# of a large map containing muliple loops)
base_transform = np.linalg.inv(geo.SE2_from_translation_angle([0.585 * 0, 0.0], 0))

all_traj_tfs = []
all_traj_test = {}
all_int_traj_test = {}

# Compute the transforms of those trajectories for plotting
for i,traj in enumerate(all_trajectories):
    int_tfs_traj = []
    for el in traj:
        if el is not None:
            int_tfs_traj.append(dw.SE2Transform.from_SE2(geo.SE2.multiply(base_transform, el)))
        else:
            int_tfs_traj.append(None)
    all_traj_tfs.append(int_tfs_traj)
    all_traj_test.update({i: int_tfs_traj})

all_traj_tfs = np.asarray(all_traj_tfs).T.tolist()

# Compute the transforms of those trajectories for plotting
for i,traj in enumerate(all_trajectories):
    cur_traj = []
    cur_traj.append(traj)
    cur_int_trajs = get_interpolated_points(center_line_global, cur_traj)
    all_int_tfs = []
    int_tfs_traj = []
    for k,traj_i in enumerate(cur_int_trajs):
        
        for el in traj_i:
            if el is not None:
                int_tfs_traj.append(dw.SE2Transform.from_SE2(geo.SE2.multiply(base_transform, el)))
            else:
                int_tfs_traj.append(None)
        all_int_tfs.append(int_tfs_traj)
        
    all_int_traj_test.update({i: int_tfs_traj})

# Compute the interpolated trajectories    
int_trajs = get_interpolated_points(center_line_global, all_trajectories)

# Compute the transforms of those trajectories for plotting
all_int_tfs = []
for i,traj in enumerate(int_trajs):
    int_tfs_traj = []
    for el in traj:
        if el is not None:
            int_tfs_traj.append(dw.SE2Transform.from_SE2(geo.SE2.multiply(base_transform, el)))
        else:
            int_tfs_traj.append(None)
    all_int_tfs.append(int_tfs_traj)
    


Now we have all the data ready to calculate the trajectory statistics.
The variable names are pretty self explenatory but here is a short description:
* mean_tfs: average trajecory computed over all the different experiments
* mean_y: Vector with all the mean offsets calculated for each position within the loop
* mean_heading: Vector with all the mean heading angles calculated for each position within the loop
* std_y: Vector with all the std of the offset calculated for each position within the loop
* std_angle: Vector with all the std of the heading angle calculated for each position within the loop
* cv_y: Vector with all the CV factors (std/mean) of the offset calculated for each position within the loop
* cv_heading: Vector with all the CV factors (std/mean) of the heading angle calculated for each position within the loop

Note CV stands for Coefficient of Variation and it helps quantifying the dispersion of the data.

In [ ]:
trajectory_info = {}

mean_tfs, std_y, std_angle, start_idx, end_idx, cv_y, cv_heading, mean_y, mean_heading \
= get_trajectories_statistics(all_int_tfs,center_line_global_tfs)

# save all the trajectory info found
trajectory_info.update({'Mean offset':mean_y,'Std offset':std_y,'CV offset':{'all':cv_y,'mean':Average(cv_y),\
                                                                             'std':statistics.stdev(cv_y),\
                                                                             'min':min(cv_y),'max':max(cv_y)},\
                        'Mean angle':mean_heading,'Std angle':std_angle,'CV angle':{'all':cv_heading,'mean':Average(cv_heading),\
                                                                                    'std':statistics.stdev(cv_heading),\
                                                                                    'min':min(cv_heading),'max':max(cv_heading)}})



The cell below scans through the vectors `cv_y` and `cv_heading` to check no more than 30 of the entries are above 1.0
If this is the case, please run another experiment to collect more data.


If this is not the case you have collected enough data trajectory wise and you can continue running this Notebook.

In [ ]:
enough_data_traj = True
cnt_cv_y = 0
cnt_cv_heading = 0
for i in cv_y:
    if i > 1.0:
        cnt_cv_y += 1        
for i in cv_heading:
    if i > 1.0:
        cnt_cv_heading += 1

#checks if at least 75% of all calculated CV are below 1
if cnt_cv_y > len(cv_y)*0.25:
    enough_data_traj = False
    print("Std of the lateral offset is too large which means that the trajectory is not stable enough, and we can not be sure that you code just had a lucky run or not." )

#check if at least 25% of all calculated CV of the heading are below one
#this is less trict as the heading does vary way more then de offset and mainly the offset is important for 
#stability. However the check is still made such that not all heading measurements have a too large CV
if cnt_cv_heading > len(cv_heading)*0.75:
    enough_data_traj = False        
    print("Std of the heading is too large which means that the trajectory is not stable enough, and we can not be sure that you code just had a lucky run or not.")
    
if enough_data_traj:
    print("Trajectory wise you have collected enough data, however there are a couple of more things to check.\n So please keep running the upcoming cells.")

Below the mean trajectory of all your experiments is plotted in red circles.

Also all the interpolated trajectories of the different experiments are ploted in other colors in circles that are a bit smaller than the ones of the overall mean trajectory.

This visualization can help you understand how "stable" you trajectory data is.

In [ ]:
#load the map
del m
m = dw.load_map(Map_Name)

# colors in which the different trajectories will be drawn
colors = ['green','cyan','blue','yellow','magenta']
# draw all the different trajectories
for k in range(0, nb_bm_found):
    current_traj_tfs = all_int_traj_test[k]
    for i, meant_tf in enumerate(current_traj_tfs):
        if not(i%2):
            if current_traj_tfs[i] != None:
                a = k%5
                m.set_object('Data-%s'% k+str(i + 10000), Circle(0.008, color=colors[a]), ground_truth=meant_tf)
    print(names[k] + ':' + colors[a])
# draw the average trajectory of all the Benchmarks
for i, meant_tf in enumerate(mean_tfs):
    if not(i%2):
        m.set_object(str(i), Circle(0.015, color='red'), ground_truth=meant_tf)

# Draw!
ipython_draw_svg(m);
    
# Save the image
outdir_im = path.join('/home/linuslingg/out', "ipython_draw_svg", "%s" % id(m))

Next the remaining data is checked. This means that the following results are compared to each other to check if the results of your Benchmark are stable enough or not. 

* Number_of_tiles_covered
* Avg_time_needed_per_tile
* mean_diff_btw_estimation_and_ground_truth_offset
* mean_diff_btw_estimation_and_ground_truth_angle
* Abs_Measurements_db_mean_offset
* Abs_Measurements_db_mean_angle
* Abs_Ground_truth_wt_mean_offset_interp
* Abs_Ground_truth_wt_mean_angle_interp
* Abs_Ground_truth_wt_mean_offset_non_interp
* Abs_Ground_truth_wt_mean_angle_non_interp

Based on this analysis the decision is made if another experiment run has to be done or if enough data has been collected to make a meaningful conclusion.

To visualize this analysis so the user gets a better understanding of why he has enough data or not, the boxplots as well as the the Error bar plots are shown below.

In [ ]:
# Plots mean and Standard Deviation 
mean_bm = {'Benchmark_Type': all_logs[0][0]['Benchmark_Type']}
std_bm = {'Benchmark_Type': all_logs[0][0]['Benchmark_Type']}
final_results = {'Benchmark_Type': all_logs[0][0]['Benchmark_Type']}
all_results_gen = {}
all_results_mean = {}
all_results_std = {}
nb_of_oos = 0
nb_of_ts = 0
enough_data_res = []

# Prepare figure for the plots, on one hand the Error bar plots and on the other hand the Boxplots
fig1, axes1 = plt.subplots(nrows=ceil(len(meaningful_comp_results)/2), ncols=2)
fig2, axes2 = plt.subplots(nrows=ceil(len(meaningful_comp_results)/2), ncols=2)
fig1.subplots_adjust(hspace=0.5)
fig1.suptitle('Error bar plot', y=.895)
fig2.subplots_adjust(hspace=0.5)
fig2.suptitle('Boxplot', y=.895)

#Saves the number of experiments that were run in the dictionary.
mean_bm.update({'Number of tests ran': number_of_tests})
final_results.update({'Number of tests ran': number_of_tests})
#Saves the actual length of the different experiments that were run in the dictionary.
final_results.update({'runtime': runtime})

#Counter for the plots
count = 0

#Scroll through all the different results stored within the files
for i, prop in enumerate(all_logs[0][0]['Results']):
    results = []
    current_prop = {}
    std_exist = False
    keep_orig = False
    save_just_col = False
    overall_traj_inf_bool = False
    #extracts the results from the different runs
    for j in range(0, nb_bm_found):
            results.append(all_logs[j][0]['Results'][prop])
    
    # Saves all the results found in the dictionary 'current_prop'
    current_prop.update({'All results': results})
    # Get the unit of the current prop we are looking at
    unit = get_unit(prop)
    
    #checks if the current property is one that is not yet implemented (Ex. marked with 'ToDo')
    if (prop not in list_of_comp_todo): 
        #Checks if the current property is in the list of properties that do not have a meaningful
        #relative comparison, respectively which have booleans stored
        if prop in no_meaningful_rel_comp:
            #those variables are not applicable as boolean results
            std_result = 'N/A'
            max_result = 'N/A'
            min_result = 'N/A'
            cv_result = 'N/A'
            median_result = 'N/A'
            #returns the number of 'True's found within the results, which helps counting the number of crashes
            counter = count_true(results)
            avg_result = '{} failures'.format(counter)
        #Checks if the current proposition is within either the list of general information or the list 
        #meaningful_comp_results (details of those lists can be found above)
        elif prop in meaningful_comp_results or prop in general_info:
            #Calculates the average of a list
            avg_result = Average(results)
            #Calculate the median, std, max and min of the results from the current property
            median_result = statistics.median(results)
            std_result = statistics.stdev(results)
            max_result = max(results)
            min_result = min(results)
            
            # coefficient of variation calculation:
            # if lower than 1, the std can be considered small enough and one can stop running tests
            # https://www.researchgate.net/post/What_do_you_consider_a_good_standard_deviation
            # First checks if the avg is different from 0 to avoid a division by 0
            # If avg is 0 and std is 0 the cv is still 0
            # However, if avg is 0 and std is defferent from 0 then the CV is large
            if avg_result != 0:
                cv_result = std_result/avg_result
                cv_median = std_result/median_result
            else:
                if std_result == 0:
                    cv_result = 0
                else:
                    cv_result = 100
            
            #if current property is in meaningful_comp_results we want to check if the std is low enough
            #this is why in this case we set std_exist = True
            if prop in meaningful_comp_results:
                std_exist = True
        
        #if the current property is in the tolerances list we just keep the original information as those 
        # should and do not change over the different experiments
        elif prop in tolerances:
            keep_orig = True
            cur_orig = results[0]
        
        #if the current property is within the list seen below, we just save the collected data and do not analyse it
        elif prop in ('begin_time_stamp_wt','Number_of_completed_laps'):
            save_just_col = True
        
        # Checks if the current property is within the list raw_traj_info
        elif prop in raw_traj_info:
            # if the current property is within the list overall_traj_info, we calculate if possible (hence if data is
            # available and it is not written 'Not available') the statistics over all the experiments ran
            # also we set the boolean 'overall_traj_inf_bool' equal True.
            if prop in overall_traj_info:
                if results[0] == 'Not available':
                    save_just_col = True
                else:
                    results  = np.hstack(results)
                    
                    overall_traj_inf_bool = True
                    avg_result = float(np.round(np.mean(results),6))
                    median_result = float(np.round(np.median(results),6))
                    std_result = float(np.round(np.std(results),6))
                    max_result = float(np.round(max(results),6))
                    min_result = float(np.round(min(results),6))
                    
                    if avg_result != 0:
                        cv_result = float(np.round(std_result/avg_result,6))
                        cv_median = float(np.round(std_result/median_result,6))

                    else:
                        if std_result == 0:
                            cv_result = 0.0
                        else:
                            cv_result = 100.0
                            
            #if the current property is not in the list overall_traj_info, we just save all the collected data
            #without further analysing it
            else:
                save_just_col = True
                
    # Save ToDo in the dictionary if the value of the current property is 'ToDo'
    else:
        keep_orig = True
        cur_orig = 'ToDO'

    #if the standard deviation exist, we check if it is small enough for the current property
    #for this we check the coefficient of variation (if it is lower 1 or not)
    #If for one property this CV is larger than 1 this property will be marked red within the plots (otw green)
    # and the user is asked to collect more data
    if std_exist:
        if cv_result >= 1.0:
            color = 'red'
            answer = 'no'
            enough_data_res.append(False)
        elif cv_result < 1.0:
            color = 'green'
            answer = 'yes'
            enough_data_res.append(True)
        else:
            color = 'black'
            answer = 'Sth went wrong'
            enough_data_res.append(False)
    
    # checks if there was some analysis of the data done above or not
    if not save_just_col and not keep_orig:
        #if there was it checks if we care about the standard deviation of this property or not
        if std_exist:
            #if we do, we update the dictionnary and we plot the results (Boxlot and Error bar plot).
            current_prop.update({'Mean': avg_result, 'Std': std_result, 'coefficient of variation calculation': cv_result,\
                                 'Median': median_result, 'Max': max_result, 'Min': min_result, 'Unit': unit,\
                                 'Enough tests run for meaningful BM analyzis': answer})
            
            # Figure 1: Mean and Std
            ax1 = axes1.flatten()[count]
            ax1.set_ylabel(unit)
            ax1.set_xlabel(prop)
            ax1.set_xticklabels([])
            ax1.errorbar(1, avg_result, yerr=std_result, linestyle='None', marker='^', ecolor = color)
            ax1.xaxis.label.set_color(color)
            # Figure 2: Boxplot:
            ax2 = axes2.flatten()[count]
            ax2.set_ylabel(unit)
            ax2.set_xlabel(prop)
            test = [1.0,2.0,3.0,4.0]
            ax2.boxplot(results, showmeans = True, notch=False, patch_artist=True,boxprops=dict(color=color,facecolor=(0.5,0.5,0.5,0.5)))
            ax2.set_xticklabels([])
            ax2.xaxis.label.set_color(color)
            #Note if the CV of the current property is too large the property will be marked red otw. green
            y = results
            x = np.random.normal(1, 0.004, size=len(y))
            ax2.plot(x, y, 'ro', alpha=0.5)
            x = np.random.normal(1, 0.0, size=len(y))
            ax1.plot(x, y, 'ro', alpha=0.5)

            count += 1
            
            all_results_std.update({'Std of ' + prop: std_result})
            
            
        else:
            #if no std exists, respectively if we don't care about the stv we just save the found data in the dictionary
            current_prop.update({'Mean': avg_result, 'Std': std_result, 'coefficient of variation calculation': cv_result,\
                                 'Median': median_result, 'Max': max_result, 'Min': min_result, 'Unit': unit})
    
    #update the dictionary all_results_mean
    all_results_mean.update({prop: current_prop})
    #checks if we just want to save the original data or also the things analysed
    if not keep_orig:
        all_results_gen.update({prop: current_prop})
    else:
        #saves the original data with its unit
        all_results_gen.update({prop+' ['+unit+']': cur_orig})

#updates the dictioaries
std_bm.update( {"Results" : all_results_std} )
mean_bm.update( {"Results" : all_results_mean} )


fig1.set_figheight(40)
fig1.set_figwidth(15)
fig2.set_figheight(40)
fig2.set_figwidth(15)

#Saves the plots
fig1.savefig(benchmark_std_graph, dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)
fig2.savefig(benchmark_boxplot_graph, dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

#saves the yaml file including all the std analysis.
with open(benchmark_std, 'w') as yaml_file:
    yaml.dump(std_bm, yaml_file, default_flow_style=False)
    


In [ ]:
if any(enough_data_res) == False:
    enough_data_res_fin = False
    print("unfortunately you did not collect enough data yet please run another experiment, analyse the date with \
    the notebook 95, add the results to the folder and then run this notebook from the beginning again")
else:
    enough_data_res_fin = True
    print("You collected enough data and can now continue withe the cell below to analyse the engineering\
    data and save all the results found")
    

If you get a green light of all different parts (meaning that the name is colored in green), please run the cell below to save the overall mean of the results to a yamle file with the name `BAGNAME_benchmark_final_results_wo_eng_data.yaml`.

If you do not get a green light and still have some red colored names in the graphs above please run another experiment as the variation of the results is too high and nothing can really be said about the results.

In [ ]:
# updates the bag
final_results.update({'Enough data traj wise': enough_data_traj})
final_results.update({'Enough data results wise': enough_data_res_fin})
final_results.update({'Results': all_results_gen})
final_results.update({'Trajectory info': trajectory_info})

In [ ]:
with open(benchmark_final_results_wo_eng_data, 'w') as yaml_file:
    yaml.dump(final_results, yaml_file, default_flow_style=False)

# Analyze Engineering Data


Please run cell by cell following the instructions.

First load the .yaml file that was computed by the hw_check you ran earlier. For this, change the variable hw_check_yaml_name below to the name your file has (Ex. CH_ETHZ_Linus_2020-05-05_DB18_hardware-compliance)

In [ ]:
hw_check_yaml_name = ''

In [ ]:
# if the file of the hardware check is at a different location, please change the logs_path below
experiment_dir = ''
outdir = path.join(logs_path_save, 'out')
hw_check_yaml_file = r'../data/'+Benchmark+'/' + hw_check_yaml_name + '.yaml'
# Dictionary containing all the information
eng_data_all = {}
dict_total_cnst = {}
static_things = {}

# dictionary in which the hw info will be saved
hw_info = {'item': 'documentation'}
#load the file and save it in the dictionary
with open(hw_check_yaml_file) as file:
    documents = yaml.full_load(file)
    hostname = documents["hostname"]
    for item, doc in documents.items():
        hw_info.update({item: doc})
        print(item, ":", doc)

# save the data found
eng_data_all.update({'HW_info': hw_info}) 

#extract the hostname
hostname_minus = hostname+'-'
hostname_underline = hostname+'_'

If you did not record a bag on the Duckiebot, please set the variable `db_data` below to `False`. If you did record a bag on the Duckiebot at least once and you successfully ran rosbag analysis with it set it to `True`.

If you did not run the Diagnostic toolbox during any experiment please set the variable `diag_toolbox_data` below to `False`. If you did run the diagnostic toolbox at least once please set it to `True`.

In [ ]:
db_data = True
diag_toolbox_data = True

In [ ]:
#update the dictionary containing all the final results
final_results.update({'db_data': db_data})
final_results.update({'diag_toolbox_data': diag_toolbox_data})


if (not db_data) and (not diag_toolbox_data):
    print("Saving the data that was found, you can stop running this notebook now and start the final comparison\
    notebook called: 96_..")
    save_data()
elif not db_data and diag_toolbox_data:
    print("Skip all the cells and jump straight to the cell with the title: Diagnostic toolbox analysis ")
elif not diag_toolbox_data and db_data:
    print("Run all the cells up to the cell with the title: Diagnostic toolbox analysis then stop")
elif diag_toolbox_data and db_data:
    print("Run all the cells up to the cells up until the end of the notebook")

Please note, that if somehow the analyze_rosbag did not generate one of the `.json` files mentioned in the upcoming cells, please just skip the cells corresponding to that `.json` file and continue with the one after.

Run the next cell such that the upload button appears. Then click on the button and select at least one BAGNAME_duckiebot_segment_count.json file that you should find within the folder data/BenchmarkXY/json. If you have run several experiments and have therefore several files please select all of them!
Make sure that after selecting the file, you click into the cell below the upload button before continuing to run.



In [ ]:
segment_count = FileUpload(accept='.json',
    multiple=True)
segment_count

Make sure you click into the cell below after you selected the correct .json file.

In [ ]:
assert segment_count.data, 'File missing, please upload in above cell'
all_data_dict = {}
all_seg_cnt = {}
overall_all_segment_count = []
start_rec_time = []
#checks how many files that were uploaded
nb_files = len(segment_count.value)
#counts the number of messages found 
nb_segm_cnts = len(json.loads(segment_count.data[0].decode('utf-8'))['meas'])



# This loads all the file and saves number of segments into the eng_data_all dictionary 
for i in range(0, len(segment_count.value)):
    all_segment_count = []
    cnt_valid_meas = 0
    dict_total_segment_count = {}
    data = json.loads(segment_count.data[i].decode('utf-8'))
    all_data_dict.update({i: data})
    start_rec_time.append(data['time'][0])
    for j in range(0,len(data['meas'])):
        time_stamp = data['time'][j]
        if time_stamp-float(start_rec_time[i]) > float(runtime[i]):
            break
        
        cnt_valid_meas += 1
        dict_cur = {'Segments': data['meas'][j]}
        all_segment_count.append(data['meas'][j])
        overall_all_segment_count.append(data['meas'][j])
        
        #uncomment the following line, if you want to save the data of each file and not just the mean and the std of them
        #dict_total_segment_count.update({time_stamp: dict_cur})
    
    #change the format to float
    all_segment_count_fl = [float(k) for k in all_segment_count]    

    # Calculate the mean, median, min, max and std of the latencies measured at different time stamps of current file.
    mean_segment_count = statistics.mean(all_segment_count_fl)
    median_segment_count = statistics.median(all_segment_count_fl)
    min_segment_count = min(all_segment_count_fl)
    max_segment_count = max(all_segment_count_fl)   
    
    #checks if we have enough data to calculate the std and cv
    if nb_segm_cnts > 1 and cnt_valid_meas > 1:
        std_segment_count = statistics.stdev(all_segment_count_fl)
        if mean_segment_count != 0:
            cv_segment_count = std_segment_count/mean_segment_count
        else:
            if std_segment_count == 0:
                cv_segment_count = 0
            else:
                cv_segment_count = 10
    else: 
        std_segment_count = 'N/A as only once a segment count published'
        cv_segment_count = 'N/A as only once a segment count published'
        
    dict_total_segment_count.update({'mean segment count': mean_segment_count, 'median segment count': median_segment_count, \
                           'std segment count': float(np.round(std_segment_count,4)),'CV segment count': float(np.round(cv_segment_count,4)),\
                           'min segment count': min_segment_count, 'max segment count': max_segment_count})

    # Add all the information to the eng_data_all dictionary if wanted
    all_seg_cnt.update({'Segment count of file ' + str(i): dict_total_segment_count})    

#change the format    
overall_all_segment_count_fl = [float(k) for k in overall_all_segment_count]    

# Calculate the mean, median, min, max and std of the latencies measured at different time stamps of current file.
overall_mean_segment_count = statistics.mean(overall_all_segment_count_fl)
overall_median_segment_count = statistics.median(overall_all_segment_count_fl)
overall_min_segment_count = min(overall_all_segment_count_fl)
overall_max_segment_count = max(overall_all_segment_count_fl)
if nb_files > 1 or (nb_segm_cnts > 1 and cnt_valid_meas > 1):
    overall_std_segment_count = statistics.stdev(overall_all_segment_count_fl)
    if overall_mean_segment_count != 0:
        overall_cv_segment_count = overall_std_segment_count/overall_mean_segment_count
    else:
        if overall_std_segment_count == 0:
            overall_cv_segment_count = 0
        else:
            overall_cv_segment_count = 10
    
else:
    overall_std_segment_count = 'N/A as only one file uploaded'
    overall_cv_segment_count = 'N/A as only one file uploaded'

#saves the date in the dictionary
all_seg_cnt.update({'Overall mean segment count': overall_mean_segment_count, \
                'Overall median segment count': overall_median_segment_count, \
                'Overall std segment count': float(np.round(overall_std_segment_count,4)), \
                'Overall CV segment count': float(np.round(overall_cv_segment_count,4)),\
                'Overall min segment count': overall_min_segment_count, \
                'Overall max segment count': overall_max_segment_count})
    
eng_data_all.update({'Segment Count': {'Nb of files': nb_files, 'Nb of segment count meas. per file': nb_segm_cnts, \
                                 'Measurements': all_seg_cnt}})


Run the next cell such that the upload button appears. Then click on the button and select at least one  BAGNAME_duckiebot_latencies.json file that you should find within the folder data/BenchmarkXY/json. If you have run several experiments and have therefore several files please select all of them! Make sure that after selecting the file, you click into the cell below the upload button before continuing to run.

In [ ]:
latency = FileUpload(accept='.json',
    multiple=True)
latency

Make sure you click into the cell below after you selected the correct .json file.

In [ ]:
assert latency.data, 'File missing, please upload in above cell'
all_data_dict = {}
all_lat = {}
overall_all_latencies = []

#checks how many files that were uploaded
nb_files = len(latency.value)
#checks how many latencies that were measured
nb_lat_meas = len(json.loads(latency.data[0].decode('utf-8'))['meas'])

# This loads all the file and saves latencies into the eng_data_all dictionary 
# This is the total lag up to and including the detector node, it is measured in ms at different time stamps
for i in range(0, len(latency.value)):
    all_latencies = []
    cnt_valid_meas = 0
    dict_total_lat = {}
    #load the data
    data = json.loads(latency.data[i].decode('utf-8'))
    all_data_dict.update({i: data})
    for j in range(0,len(data['meas'])):
        time_stamp = data['time'][j]
        # This should be stoped if the time where the latency was published
        # is after the crash occured, compare this time to the 
#         if time_stamp-float(start_rec_time[i]) > float(runtime[i]):
#             break
        cnt_valid_meas += 1
        dict_cur = {'latency': data['meas'][j] + " ms"}
        all_latencies.append(data['meas'][j])
        overall_all_latencies.append(data['meas'][j])
        
        #uncomment the following line, if you want to save the data of each file and not just the mean and the std of them
        #dict_total_lat.update({time_stamp: dict_cur})
    
    #change the format
    all_latencies_fl = [float(k) for k in all_latencies]    

    # Calculate the mean, median, min, max and std of the latencies measured at different time stamps of current file.
    mean_latency = statistics.mean(all_latencies_fl)
    median_latency = statistics.median(all_latencies_fl)
    min_latency = min(all_latencies_fl)
    max_latency = max(all_latencies_fl)
    
    #checks if we have enough data to calculate std and cv
    if nb_lat_meas > 1 and cnt_valid_meas > 1:
        std_latency = float(np.round(statistics.stdev(all_latencies_fl),4))
        if mean_latency != 0:
            cv_lat = float(np.round(std_latency/mean_latency,4))
        else:
            if std_latency == 0:
                cv_lat = 0
            else:
                cv_lat = 10
    else: 
        std_latency = 'N/A as only one valid latency measured'
        cv_lat = 'N/A as only one valid latency measured'
        
    dict_total_lat.update({'mean latency (ms)': mean_latency, 'median latency (ms)': median_latency, \
                           'std latency (ms)': std_latency,'CV latency': cv_lat,\
                           'min latency (ms)': min_latency, 'max latency(ms)': max_latency})

    # Add all the information to the eng_data_all dictionary if wanted
    all_lat.update({'Latency of file ' + str(i): dict_total_lat})    

#change the format    
overall_all_latencies_fl = [float(k) for k in overall_all_latencies]    



#checks if we have enough measurements to calculate the std and cv
if (nb_files > 1 and cnt_valid_meas > 1) or (nb_lat_meas > 1 and cnt_valid_meas > 1) :
    # Calculate the mean, median, min, max and std of the latencies measured at different time stamps of current file.
    overall_mean_latency = statistics.mean(overall_all_latencies_fl)
    overall_median_latency = statistics.median(overall_all_latencies_fl)
    overall_min_latency = min(overall_all_latencies_fl)
    overall_max_latency = max(overall_all_latencies_fl)
    overall_std_latency = float(np.round(statistics.stdev(overall_all_latencies_fl),4))
    if overall_mean_latency != 0:
        overall_cv_lat = float(np.round(overall_std_latency/overall_mean_latency,4))
    else:
        if overall_std_latency == 0:
            overall_cv_lat = 0
        else:
            overall_cv_lat = 10
    
else:
    # Calculate the mean, median, min, max and std of the latencies measured at different time stamps of current file.
    overall_mean_latency = overall_all_latencies_fl
    overall_median_latency = overall_all_latencies_fl
    overall_min_latency = min(overall_all_latencies_fl)
    overall_max_latency = max(overall_all_latencies_fl)
    overall_std_latency = 'N/A as only one file uploaded'
    overall_cv_lat = 'N/A as only one file uploaded'

all_lat.update({'Overall mean latency (ms)': overall_mean_latency, 'Overall median latency (ms)': overall_median_latency, \
                'Overall std latency (ms)': overall_std_latency, 'Overall CV latency': overall_cv_lat,\
                'Overall min latency (ms)': overall_min_latency, 'Overall max latency(ms)': overall_max_latency})
    
eng_data_all.update({'Latency': {'Nb of files': nb_files, 'Nb of latency meas. per file': nb_lat_meas, \
                                 'Measurements': all_lat}}) 

Run the next cell such that the upload button appears. Then click on the button and select at least one BAGNAME_duckiebot_node_info.json file that you should find within the folder data/BenchmarkXY/json. 
If you have run several experiments and have therefore several files please select all of them! 
Make sure that after selecting the file, you click into the cell below the upload button before continuing to run.

In [ ]:
node_inf = FileUpload(accept='.json',
    multiple=True)
node_inf

Make sure you click into the cell below after you selected the correct .json file.


In [ ]:
assert node_inf.data, 'File missing, please upload in above cell'

all_data_dict = {}
dict_total = {}
all_nodes = {}
#checks how many nodes that were recorded
nb_nodes = len(json.loads(node_inf.data[0].decode('utf-8'))['node'])
#checks how many files that were uploaded
nb_files = len(node_inf.value)

#extract the uploaded information
for i in range(0, len(node_inf.value)):
    data = json.loads(node_inf.data[i].decode('utf-8'))
    all_data_dict.update({i: data})
    # This loads the file and saves all the informations about the nodes into the eng_data_all dictionary 
    for j in range(0,len(data['node'])):
        dict_cur = {'frequency (Hz)': float(np.round(data['frequency'][j],4)), 'message_count': float(np.round(data['message_count'][j],4)), \
                    'connections': float(np.round(data['connections'][j],4))}
        node_name = data['node'][j]
        dict_total[node_name] = dict_cur
        
    #uncomment the following line, if you want to save the data of each file and not just the mean and the std of them
    #eng_data_all.update({'Node Info of file'+ str(i): dict_total})

    
if nb_files > 1:
    #if there are more then one file uploaded calculate the mean, std and cv of all of the nodes
    for i in range(0,nb_nodes):
        current ={}
        freq_all = []
        mes_cnt_all = []
        nb_connections_all = []
        for j in range(0, nb_files):
            freq_all.append(all_data_dict[j]['frequency'][i])
            mes_cnt_all.append(all_data_dict[j]['message_count'][i])
            nb_connections_all.append(all_data_dict[j]['connections'][i])

        current.update({'Mean frequency (Hz)': float(np.round(statistics.mean(freq_all),4)), \
                        'Std frequency (Hz)': float(np.round(statistics.stdev(freq_all),4)), \
                        'CV frequency (Hz)': float(np.round(statistics.stdev(freq_all)/statistics.mean(freq_all),4)), \
                        'Mean message_count':  float(np.round(statistics.mean(mes_cnt_all),4)), \
                        'Std message_count': float(np.round(statistics.stdev(mes_cnt_all),4)),\
                        'CV message_count': float(np.round(statistics.stdev(mes_cnt_all)/statistics.mean(mes_cnt_all),4)), \
                        'Mean connections':  float(np.round(statistics.mean(nb_connections_all),4)),\
                        'Std connections': float(np.round(statistics.stdev(nb_connections_all),4)),\
                        'CV connections': float(np.round(statistics.stdev(nb_connections_all)/statistics.mean(nb_connections_all),4))})

        all_nodes.update({all_data_dict[0]['node'][i]: current})
#  float(np.round(overall_swap_median,4))   
else: 
    #if only one file is uploaded, the one measurement is taken as mean value
    for i in range(0,nb_nodes):
        current ={}
        current.update({'Mean frequency (Hz)': float(np.round(all_data_dict[0]['frequency'][i],4)), \
                            'Std frequency (Hz)': 'N/A as only one file uploaded', \
                            'CV frequency (Hz)': 'N/A as only one file uploaded', \
                            'Mean message_count':  float(np.round(all_data_dict[0]['message_count'][i],4)), \
                            'Std message_count': 'N/A as only one file uploaded',\
                            'CV message_count': 'N/A as only one file uploaded', \
                            'Mean connections':  float(np.round(all_data_dict[0]['connections'][i],4)),\
                            'Std connections': 'N/A as only one file uploaded',\
                            'CV connections': 'N/A as only one file uploaded'})
        all_nodes.update({all_data_dict[0]['node'][i]: current})

eng_data_all.update({'Node Info': {'Nb of files': nb_files, 'Nb of nodes': nb_nodes, 'Measurements': all_nodes}}) 

Run the next cell such that the upload button appears. Then click on the button and select the BAGNAME_duckiebot_constant.json file that you should find within the folder data/BenchmarkXY/json. Make sure that after selecting the file, you click into the cell below the upload button before continuing to run.

Please only select one file. It does not matter out of which experiment as this data is static and does not change.

In [ ]:
constants = FileUpload(accept='.json',
    multiple=False)
constants

Make sure you click into the cell below after you selected the correct .json file.

In [ ]:
assert constants.data, 'File missing, please upload in above cell'
data = json.loads(constants.data[0].decode('utf-8'))


# Extracts all the constances name and value
# Constances are the values set on the duckiebot as example the gain, trim etc.
# These values don't change over the tests run for the same benchmark (hence static) and are reported in the final
# benchmark report.
for cnst_name in data:
    dict_total_cnst[cnst_name] = data[cnst_name]
    
# Constances added to the static_things dictionary
static_things.update({'Constants': dict_total_cnst})



In [ ]:
if not diag_toolbox_data:
    print("saving the data as apparently there is no data available from the diagnostic toolbox")
    save_data()

You can stop running this notebook if you do not have any data from the diagnostic toolbox colected.

## Diagnostic toolbox analysis

Run the next cell such that the upload button appears. Then click on the button and select the .json file you downloaded from the dashboard webside (created by the diagnostic toolbox) and placed within the folder data/BenchmarkXY/json. Make sure that after selecting the file, you click into the cell below the upload button before continuing to run.

In [ ]:
upload = FileUpload(accept='.json',
    multiple=True)
upload

Make sure you click into the cell below after you selected the correct .json file.

In [ ]:
assert upload.data, 'File missing, please upload in above cell'

all_data_dict = {}
dict_total = {}
overall_cpu = []
overall_swap = []
overall_memory = []

#checks how many files that were uploaded
nb_files = len(upload.value)

for i in range(0, len(upload.value)):
    data = json.loads(upload.data[i].decode('utf-8'))
    all_data_dict.update({i: data})
    total_current = {}
    meas_name = ['Memory', 'Swap', 'CPU']
    bm_data = np.array([[],[],[],[]])
    t0= data['resources_stats'][0]['time']
    all_data = []
    # Extracts the information about the overall Memory, Swap and CPU usage over time
    for j,meas in enumerate(data['resources_stats']):
        # Only saves/analyses the data that is collected before the crash occured (if a crash occured)
        if float(meas['time'])-float(t0) > float(runtime[i]):
            last_indx = j-1
            dat = np.array([[meas['time']-t0, meas['memory']['used']/meas['memory']['total']*100, \
                             meas['swap']['used']/meas['swap']['total']*100, meas['cpu']['pcpu']]])
            tot_mem_max = meas['memory']['total']
            tot_swap_max = meas['swap']['total']
            all_data.append(dat.T)
            bm_data = np.append(bm_data, dat.T, axis=1)
            break
        dat = np.array([[meas['time']-t0, meas['memory']['used']/meas['memory']['total']*100, \
                         meas['swap']['used']/meas['swap']['total']*100, meas['cpu']['pcpu']]])
        tot_mem_max = meas['memory']['total']
        tot_swap_max = meas['swap']['total']
        all_data.append(dat.T)
        bm_data = np.append(bm_data, dat.T, axis=1)

    cpu_total = bm_data[3]
    swap_total = bm_data[2]
    memory_total = bm_data[1]
    overall_cpu.append(bm_data[3])
    overall_swap.append(bm_data[2])
    overall_memory.append(bm_data[1])

    total_cpu_mean = np.mean(cpu_total)
    total_cpu_median = np.median(cpu_total)
    total_cpu_std = np.std(cpu_total)
    if total_cpu_mean != 0:
        total_cpu_cv = total_cpu_std/total_cpu_mean
    else:
        if total_cpu_std == 0:
            total_cpu_cv = 0
        else:
            total_cpu_cv = 10
    total_cpu_cv = total_cpu_std/total_cpu_mean
    total_swap_mean = np.mean(swap_total)
    total_swap_median = np.median(swap_total)
    total_swap_std = np.std(swap_total)
    if total_swap_mean != 0:
        total_swap_cv = total_swap_std/total_swap_mean
    else:
        if total_swap_std == 0:
            total_swap_cv = 0
        else:
            total_swap_cv = 10
    total_memory_mean = np.mean(memory_total)
    total_memory_median = np.median(memory_total)
    total_memory_std = np.std(memory_total)
    if total_memory_mean != 0:
        total_memory_cv = total_memory_std/total_memory_mean
    else:
        if total_memory_std == 0:
            total_memory_cv = 0
        else:
            total_memory_cv = 10
    

    total_current = {'CPU (CPU used in %)': {'Mean' : float(np.round(total_cpu_mean,4)), 'Median' : float(np.round(total_cpu_median,4)),\
                           'Standard Deviation' : float(np.round(total_cpu_std,4)), 'coefficient of variation': float(np.round(total_cpu_cv,4))}, \
                   'Swaps': {'Mean' : float(np.round(total_swap_mean,4)), 'Median' : float(np.round(total_swap_median,4)),\
                           'Standard Deviation' : float(np.round(total_swap_std,4)), 'coefficient of variation': float(np.round(total_swap_cv,4)),\
                            'Swaps max': float(np.round(tot_swap_max,4))}, \
                   'Mem (memory used in %)': {'Mean' : float(np.round(total_memory_mean,4)), 'Median' : float(np.round(total_memory_median,4)),\
                           'Standard Deviation' : float(np.round(total_memory_std,4)), 'coefficient of variation': \
                           float(np.round(total_memory_cv,4)), 'Memory Max (bytes)': float(np.round(tot_mem_max,4))}}

    dict_total.update({'Engineering data of file ' + str(i): total_current})
    
    
    #Below there is the code that plots all the data so you can have a look at the behaviour of the total CPU, 
    #NThreads and Memory over the time in which the diagnostic toolbox (and therefore the Benchmark) was running. 
    #If you run the cell, you will see a plot an interpolated line of the total CPU usage, NThreads and Memory 
    #usage.
    time_ip = np.linspace(bm_data[0][0], bm_data[0][-1], 100)
    bm_ip = np.array([time_ip])
    fig, axes= plt.subplots(3, 1, figsize=(9, 8))
    fig.text(0.5, 0.04, 'time', ha='center', va='center')
    fig.text(0.03, 0.5, 'Performance', ha='center', va='center', rotation='vertical')
    
    for k in range(len(bm_data)-1):
        tck = interpolate.splrep(bm_data[0], bm_data[k+1], s=0)

        ip = np.array([interpolate.splev(bm_ip[0], tck, der=0)])
        bm_ip = np.append(bm_ip, ip, axis=0)

        axes[k].plot(bm_data[0], bm_data[k+1], bm_ip[0], bm_ip[k+1])
        axes[k].legend(['Measurement', 'IP Measurement'])
        axes[k].set_title(meas_name[k])
        axes[k].set_ylim(0, 100)
        axes[k].set_ylabel('%')

        fig.suptitle('Diagnostics', fontsize=16)
    plt.show()

current = {}
if nb_files > 1:
    #if more than one file is uploaded we calculate the overall mean,stv and cv
    all_cpu = []
    for i in range(0,nb_files):
        for j in range (0, len(overall_cpu[i])):
            all_cpu.append(overall_cpu[i][j])
    overall_cpu_mean = np.mean(all_cpu)
    overall_cpu_median = np.median(all_cpu)
    overall_cpu_std = np.std(all_cpu)
    if overall_cpu_mean != 0:
        overall_cpu_cv = overall_cpu_std/overall_cpu_mean
    else:
        if overall_cpu_std == 0:
            overall_cpu_cv = 0
        else:
            overall_cpu_cv = 10
    all_swap = []
    for i in range(0,nb_files):
        for j in range (0, len(overall_swap[i])):
            all_swap.append(overall_swap[i][j])
    overall_swap_mean = np.mean(all_swap)
    overall_swap_median = np.median(all_swap)
    overall_swap_std = np.std(all_swap)
    if overall_swap_mean != 0:
        overall_swap_cv = overall_swap_std/overall_swap_mean
    else:
        if overall_swap_std == 0:
            overall_swap_cv = 0
        else:
            overall_swap_cv = 10
    all_mem = []
    for i in range(0,nb_files):
        for j in range (0, len(overall_memory[i])):
            all_mem.append(overall_memory[i][j])
    overall_memory_mean = np.mean(all_mem)
    overall_memory_median = np.median(all_mem)
    overall_memory_std = np.std(all_mem)
    if overall_memory_mean != 0:
        overall_memory_cv = overall_memory_std/overall_memory_mean
    else:
        if overall_memory_std == 0:
            overall_memory_cv = 0
        else:
            overall_memory_cv = 10

    current = {'Overall CPU (CPU used in %)': {'Mean' : float(np.round(overall_cpu_mean,4)), \
                                                     'Median' : float(np.round(overall_cpu_median,4)),\
                                                     'Standard Deviation' : float(np.round(overall_cpu_std,4)),\
                                                     'coefficient of variation': float(np.round(overall_cpu_cv,4))},\
                   'Overall Swaps (Swaps used in %)': {'Mean' : float(np.round(overall_swap_mean,4)),\
                                     'Median' : float(np.round(overall_swap_median,4)),\
                                     'Standard Deviation' : float(np.round(overall_swap_std,4)),\
                                     'coefficient of variation': float(np.round(overall_swap_cv,4)),\
                                     'Swaps max': float(np.round(tot_swap_max,4))},\
                   'Overall Mem (memory used in %)': {'Mean' : float(np.round(overall_memory_mean,4)),\
                                                      'Median' : float(np.round(overall_memory_median,4)),\
                                                      'Standard Deviation' : float(np.round(overall_memory_std,4)),\
                                                      'coefficient of variation':float(np.round(overall_memory_cv,4)),\
                                                      'Memory Max (bytes)': float(np.round(tot_mem_max,4))}}


    dict_total.update({'Overall engineering data container': current})
    
else:
    #if only one file was uploaded this one measurement is taken as mean
    overall_cpu_mean = np.mean(overall_cpu)
    overall_cpu_median = np.median(overall_cpu)
    overall_cpu_std = 'N/A as only one file uploaded'
    overall_cpu_cv = 'N/A as only one file uploaded'
    overall_swap_mean = np.mean(overall_swap)
    overall_swap_median = np.median(overall_swap)
    overall_swap_std = 'N/A as only one file uploaded'
    overall_swap_cv = 'N/A as only one file uploaded'
    overall_memory_mean = np.mean(overall_memory)
    overall_memory_median = np.median(overall_memory)
    overall_memory_std = 'N/A as only one file uploaded'
    overall_memory_cv = 'N/A as only one file uploaded'

    current = {'Overall CPU (CPU used in %)': {'Mean' : float(np.round(overall_cpu_mean,4)),\
                                                     'Median' : float(np.round(overall_cpu_median,4)),\
                                                     'Standard Deviation' : overall_cpu_std,\
                                                     'coefficient of variation': overall_cpu_cv},\
                   'Overall Swaps': {'Mean' : float(np.round(overall_swap_mean,4)),\
                                     'Median' : float(np.round(overall_swap_median,4)),\
                                     'Standard Deviation' : overall_swap_std,\
                                     'coefficient of variation': overall_swap_cv,\
                                     'Swaps max': float(np.round(tot_swap_max,4))},\
                   'Overall Mem (memory used in %)': {'Mean' : float(np.round(overall_memory_mean,4)),\
                                                      'Median' : float(np.round(overall_memory_median,4)),\
                                                      'Standard Deviation' : overall_memory_std,\
                                                      'coefficient of variation':overall_memory_cv,\
                                                      'Memory Max (bytes)': float(np.round(tot_mem_max,4))}}
    dict_total.update({'Overall engineering data container': current})

# Add all the data to the eng_data_all dictionary
eng_data_all.update({'Total engineering data container': {'Nb of files': nb_files, 'Measurements': dict_total}})


Please change the variable `dt_core_name` below to the name, under which you ran the dt-core
If you followed carefully the instructions, this should be: 'behaviour_benchmarking'.
If you ran the normal lane_following demo you should change the variable to 'lane_following'.

(If you have not given a name, it will be a random name generated by docker, please run the cell below to see all the differen names, the one with the very random name will be it as all other containers should have meaningful and proper names.
If the name of the container is different for each experiment, just use the list version of the variable called `dt_core_name_list`, note that this list should have as many entries as files uploaded)

In [ ]:
for i in range(0, len(upload.value)):
    data = json.loads(upload.data[i].decode('utf-8'))
    container_id = data['containers']
    for x in container_id:
        print(container_id[x])

In [ ]:
#if the name for dt-core is the same for all experiments, please change 'dt_core_name' below and comment out the
#line at the very end of this cell
#uncoment line below if all have same name otw comment it out
dt_core_name = 'behaviour_benchmarking'

#make a list out of it with the length of files uploaded
#uncoment lines below if all have same name otw comment it out
dt_core_name_list=[]
for i in range(0, len(upload.value)):
    dt_core_name_list.append(dt_core_name)

#if the name for dt-core is different in each experiment, please comment the lines above and uncoment the line below
#note the list below should have as many entries as you uploaded dashboard files
# dt_core_name_list = ['','','','','']

In [ ]:
all_data_dict = {}
dict_total = {}
overall_cpu_processes_fl = {}
overall_nthreads_fl = {}
overall_mem_fl = {}

for i in range(0, len(upload.value)):
    data = json.loads(upload.data[i].decode('utf-8'))

    cpu_processes = np.array([[],[],[]])
    nthreads_processes = np.array([[],[],[]])
    memory = np.array([[],[],[]])

    t0= data['process_stats'][0]['time']
    memory_max = float(np.round(data['resources_stats'][0]['memory']['total'],4))

    container_id = data['containers']

    # At the moment the containers considered are:
    # 'behaviour_benchmarking', ('dts-run-diagnostics-system-monitor'), 'acquisition-bridge', 'demo_all_drivers', 
    # 'demo_all'

    # dictionary where all container names and their corresponding keys are safed
    all_keys = {}

    # saves the key corresponding to the containers considered
    for x in container_id:
        if container_id[x] == dt_core_name_list[i]:
            behaviour_benchmarking_key = x
            all_keys.update({container_id[x]: x})
        if container_id[x] == 'dts-run-diagnostics-system-monitor': 
            diagnostics_system_monitor_key = x
            all_keys.update({container_id[x]: x})
        if container_id[x] == 'acquisition-bridge': 
            acquisition_bridge_key = x
            all_keys.update({container_id[x]: x})
        if container_id[x] == 'demo_all_drivers': 
            demo_all_drivers_key = x
            all_keys.update({container_id[x]: x})
        if container_id[x] == 'demo_all': 
            demo_all_key = x
            all_keys.update({container_id[x]: x})

    process_stats = data['process_stats']
    container_stats = data['container_stats']

    # saves the cpu usage in % (compared to the total available cpu), the number of threads as well as the memory 
    # used in % (compared to the total available memory) of the single containers 
    for x in process_stats:
        if x['container'] == behaviour_benchmarking_key:
            dat = np.array([[x['time']-t0,float(x['pcpu']),x['command']]])
            cpu_processes = np.append(cpu_processes,dat.T, axis=1)
            dat = np.array([[x['time']-t0,float(x['nthreads']),x['command']]])
            nthreads_processes = np.append(nthreads_processes,dat.T, axis=1)
            #place pmem instead of mem if wanted
            dat = np.array([[x['time']-t0,float(x['pmem']),x['command']]])
            memory = np.append(memory,dat.T, axis=1)

    command = cpu_processes[2]
    container_names = []


    length = cpu_processes.shape[1]
    occurrences = np.count_nonzero(cpu_processes == cpu_processes[0][2])
    # Commented out as same number of occurrences as cpu
    # occurrences_nthreads = np.count_nonzero(nthreads_processes == nthreads_processes[0][2])
    time_ip = np.linspace(float(cpu_processes[0][0]), float(cpu_processes[0][-1]), 100)
    bm_ip = np.array([time_ip])

    # dictionary that sumerizes the cpu, nthreads and memory used by the different containers
    summary = {'Node': {'Engineering Data Performance': 'Value'}}

    for j in range(0,occurrences-1):
        pos = np.char.find(command[j],hostname_minus)
        if pos == -1:
            pos = np.char.find(command[j],hostname_underline)
        # splits the found values (cpu, nthreads and memory) into lists of the different containers                  
        current_container_name = command[j][pos:]
        container_names.append(current_container_name)
        cpu_processes_int = cpu_processes[:,j:length-1:occurrences]
        nthreads_processes_int = nthreads_processes[:,j:length-1:occurrences]
        mem_processes_int = memory[:,j:length-1:occurrences]

        # calculates the mean, median and std of the cpu, nthreads and memory used by the different containers
        # Also the cv is calculated, this is the coefficient of variation calculation:
        # if lower than 1, the std can be considered small enough and one can stop running tests
        # https://www.researchgate.net/post/What_do_you_consider_a_good_standard_deviation
        cpu_processes_fl = cpu_processes_int[1].astype(np.float)
        if i == 0:
            overall_cpu_processes_fl[j] = cpu_processes_fl
        else:
            overall_cpu_processes_fl[j] = np.append(overall_cpu_processes_fl[j],cpu_processes_fl)
        cpu_processes_mean = np.mean(cpu_processes_fl)
        cpu_processes_median = np.median(cpu_processes_fl)
        cpu_processes_std = np.std(cpu_processes_fl)
        if cpu_processes_mean != 0:
            cpu_cv = float(np.round(cpu_processes_std/cpu_processes_mean,4))
        else:
            if cpu_processes_std == 0:
                cpu_cv = 0
            else:
                cpu_cv = 10
        nthreads_fl = nthreads_processes_int[1].astype(np.float)
        if i == 0:
            overall_nthreads_fl[j] = nthreads_fl
        else:
            overall_nthreads_fl[j] = np.append(overall_nthreads_fl[j],nthreads_fl)
        nthreads_mean = np.mean(nthreads_fl)
        nthreads_median = np.median(nthreads_fl)
        nthreads_std = np.std(nthreads_fl)
        if nthreads_mean != 0:
            nthreads_cv = float(np.round(nthreads_std/nthreads_mean,4))
        else:
            if nthreads_std == 0:
                nthreads_cv = 0
            else:
                nthreads_cv = 10
        mem_fl = mem_processes_int[1].astype(np.float)
        if i == 0:
            overall_mem_fl[j] = mem_fl
        else:
            overall_mem_fl[j] = np.append(overall_mem_fl[j],mem_fl)
        mem_mean = np.mean(mem_fl)
        mem_median = np.median(mem_fl)
        mem_std = np.std(mem_fl)
        if mem_mean != 0:
            mem_cv = float(np.round(mem_std/mem_mean,4))
        else:
            if mem_std == 0:
                mem_cv = 0
            else:
                mem_cv = 10

        # save the data in the dictionary 
        current = {'CPU (CPU used in %)': {'Mean' : float(np.round(cpu_processes_mean,4)), 'Median' : float(np.round(cpu_processes_median,4)),\
                           'Standard Deviation' : float(np.round(cpu_processes_std,4)), 'coefficient of variation': cpu_cv}, \
                   'NThreads': {'Mean' : float(np.round(nthreads_mean,4)), 'Median' : float(np.round(nthreads_median,4)),\
                           'Standard Deviation' : float(np.round(nthreads_std,4)), 'coefficient of variation': nthreads_cv}, \
                   'PMem (memory used in %)': {'Mean' : float(np.round(mem_mean,4)), 'Median' : float(np.round(mem_median,4)),\
                           'Standard Deviation' : float(np.round(mem_std,4)), 'coefficient of variation': mem_cv, \
                           'Memory Max (bytes)': float(np.round(memory_max,4))}}

        summary.update( {current_container_name: current} )

    # Add all the data to the eng_data_all dictionary
    dict_total.update({'Engineering data node of file ' + str(i): summary})

node_cur = {}

if nb_files > 1:
    #if there was more then one file uploaded the mean, std and cv over all the data is calculated
    for j in range(0,occurrences-1):
        cpu_processes_mean = np.mean(overall_cpu_processes_fl[j])
        cpu_processes_median = np.median(overall_cpu_processes_fl[j])
        cpu_processes_std = np.std(overall_cpu_processes_fl[j])
        cpu_cv = float(np.round(cpu_processes_std/cpu_processes_mean,4))
        nthreads_mean = np.mean(overall_nthreads_fl[j])
        nthreads_median = np.median(overall_nthreads_fl[j])
        nthreads_std = np.std(overall_nthreads_fl[j])
        nthreads_cv = float(np.round(nthreads_std/nthreads_mean,4))
        mem_mean = np.mean(overall_mem_fl[j])
        mem_median = np.median(overall_mem_fl[j])
        mem_std = np.std(overall_mem_fl[j])
        mem_cv = float(np.round(mem_std/mem_mean,4))
        current = {'Overall CPU (CPU used in %)': {'Mean' : float(np.round(cpu_processes_mean,4)), 'Median' : float(np.round(cpu_processes_median,4)),\
                           'Standard Deviation' : float(np.round(cpu_processes_std,4)), 'coefficient of variation': float(np.round(cpu_cv,4))}, \
                   'Overall NThreads': {'Mean' : float(np.round(nthreads_mean,4)), 'Median' : float(np.round(nthreads_median,4)),\
                           'Standard Deviation' : float(np.round(nthreads_std,4)), 'coefficient of variation': float(np.round(nthreads_cv,4))}, \
                   'Overall PMem (memory used in %)': {'Mean' : float(np.round(mem_mean,4)), 'Median' : float(np.round(mem_median,4)),\
                           'Standard Deviation' : float(np.round(mem_std,4)), 'coefficient of variation': float(np.round(mem_cv,4)), \
                           'Memory Max (bytes)': float(np.round(memory_max,4))}}
        
        node_cur.update({container_names[j]: current})
        
    
else:
    #if only one file was uploaded the only measurement is taken as mean 
    for j in range(0,occurrences-1):
        cpu_processes_mean = np.mean(overall_cpu_processes_fl[j])
        cpu_processes_median = np.median(overall_cpu_processes_fl[j])
        cpu_processes_std = 'N/A as only one file uploaded'
        cpu_cv = 'N/A as only one file uploaded'
        nthreads_mean = np.mean(overall_nthreads_fl[j])
        nthreads_median = np.median(overall_nthreads_fl[j])
        nthreads_std = 'N/A as only one file uploaded'
        nthreads_cv = 'N/A as only one file uploaded'
        mem_mean = np.mean(overall_mem_fl[j])
        mem_median = np.median(overall_mem_fl[j])
        mem_std = 'N/A as only one file uploaded'
        mem_cv = 'N/A as only one file uploaded'
        
        current = {'Overall CPU (CPU used in %)': {'Mean' : float(np.round(cpu_processes_mean,4)), 'Median' : float(np.round(cpu_processes_median,4)),\
                           'Standard Deviation' : cpu_processes_std, 'coefficient of variation': cpu_cv}, \
                   'Overall NThreads': {'Mean' : float(np.round(nthreads_mean,4)), 'Median' : float(np.round(nthreads_median,4)),\
                           'Standard Deviation' : nthreads_std, 'coefficient of variation': nthreads_cv}, \
                   'Overall PMem (memory used in %)': {'Mean' : float(np.round(mem_mean,4)), 'Median' : float(np.round(mem_median,4)),\
                           'Standard Deviation' : mem_std, 'coefficient of variation': mem_cv, \
                           'Memory Max (bytes)': float(np.round(memory_max,4))}}
        

        node_cur.update({container_names[j]: current})

dict_total.update({'Overall engineering data node': node_cur})
eng_data_all.update({'Total engineering data node': {'Nb of files': nb_files, 'Measurements': dict_total}})


In [ ]:
container_info = {'Container Name': {'Type': 'Information'}}

# Dictionary which will hold all the "static" information about the software
# This means, the container names, their tags, version etc (see below) and also the duckiebot name of the duckiebot
# used for this benchmark
# This will be used in the final benchmark report


for i, cur_key in enumerate(all_keys):

    # Not very usefull as always different
    # name = data['container_config'][demo_all_drivers_key]['Args'][1]
    image_name = data['container_config'][all_keys[cur_key]]['Config']['Image']
    image_tag = data['container_config'][all_keys[cur_key]]['Image']
    cont_name = data['container_config'][all_keys[cur_key]]['Name']
    autobot_name = data['container_config'][all_keys[cur_key]]['Config']['Hostname']
    dt_label_arch = data['container_config'][all_keys[cur_key]]['Config']['Labels']['org.duckietown.label.architecture']
    dt_label_base_img = data['container_config'][all_keys[cur_key]]['Config']['Labels']['org.duckietown.label.base.image']
    dt_code_branch = data['container_config'][all_keys[cur_key]]['Config']['Labels']['org.duckietown.label.code.branch']
    dt_code_repository = data['container_config'][all_keys[cur_key]]['Config']['Labels']['org.duckietown.label.code.repository']
    dt_code_version_major = data['container_config'][all_keys[cur_key]]['Config']['Labels']['org.duckietown.label.code.version.major']
    dt_module_type = data['container_config'][all_keys[cur_key]]['Config']['Labels']['org.duckietown.label.module.type']
    dt_template_name = data['container_config'][all_keys[cur_key]]['Config']['Labels']['org.duckietown.label.template.name']

    curr = {'Image Name': image_name, 'Image Tag': image_tag, 'Image Name and Tag': image_name + '@' + image_tag, \
            'Container Name': cont_name, 'Autobot Name': autobot_name, 'DT label architecture': dt_label_arch, \
            'DT label base image': dt_label_base_img, 'DT label code branch': dt_code_branch,\
            'DT label code repository': dt_code_repository, 'DT label code version major': dt_code_version_major,\
            'DT label module type': dt_module_type, 'DT label template name': dt_template_name}
    
    container_info.update({cur_key: curr})

    
static_things.update({'Dashboard Info': container_info})

save_data() 

dashboard_info = path.join(outdir, name + '_dashboard_info.yaml')

    
with open(dashboard_info, 'w') as yaml_file:
    yaml.dump(container_info, yaml_file, default_flow_style=False)
# print((data['container_config'][diagnostics_system_monitor_key]['Config']['Labels']['org.duckietown.label.base.image']))

Below there is a cell you can use to have a look at the behaviour of the CPU, NThreads and Memory of all the seperate containers over the time in which the diagnostic toolbox (and therefore the Benchmark) was running. If you run the cell, you will see a plot an interpolated line of CPU usage, NThreads and Memory usage for each container of the last file you uploaded. 

In [ ]:
fig, axes= plt.subplots(occurrences, 1, figsize=(15, 40))
fig.text(0.5, 0.04, 'time', ha='center', va='center')
fig.text(0.03, 0.5, 'Performance', ha='center', va='center', rotation='vertical')

# Plots of the CPU usage behaviour over time for all containers
for i in range(0,occurrences-1):
    pos = np.char.find(command[i],hostname_minus)
    if pos == -1:
        pos = np.char.find(command[i],hostname_underline)
                       
    current_container_name = command[i][pos:]
    container_names.append(current_container_name)
    cpu_processes_int = cpu_processes[:,i:length-1:occurrences]
    
    for j in range(0,len(cpu_processes)-2):
        tck = interpolate.splrep((cpu_processes_int[0]), (cpu_processes_int[j+1]), s=0)
        ip = np.array([interpolate.splev(bm_ip[0], tck, der=0)])
        time_ip = np.linspace(float(cpu_processes_int[0][0]), float(cpu_processes_int[0][-1]), 100)
        cpu_processes_int_x = np.array([time_ip])
        value_ip = np.linspace(float(cpu_processes_int[1][0]), float(cpu_processes_int[1][-1]), 100)
        cpu_processes_int_y = np.array([value_ip])
        
        
        axes[i].plot((cpu_processes_int_x[0]), cpu_processes_int_y[0])
        axes[i].legend(['Measurement', 'IP Measurement'])
        axes[i].set_title(current_container_name)
        axes[i].set_ylabel('%')
         # ToDo add units to axes
    
fig.suptitle('pcpu', fontsize=16)


fig, axes= plt.subplots(occurrences, 1, figsize=(15, 40))
fig.text(0.5, 0.04, 'time', ha='center', va='center')
fig.text(0.03, 0.5, 'Performance', ha='center', va='center', rotation='vertical')

# Plots of the nthreads used over time for all containers
for i in range(0,occurrences-1):
    nthreads_processes_int = nthreads_processes[:,i:length-1:occurrences]
    for j in range(0,len(cpu_processes)-2):
        tck = interpolate.splrep((nthreads_processes_int[0]), (nthreads_processes_int[j+1]), s=0)
        ip = np.array([interpolate.splev(bm_ip[0], tck, der=0)])
        time_ip = np.linspace(float(nthreads_processes_int[0][0]), float(nthreads_processes_int[0][-1]), 100)
        nthreads_processes_int_x = np.array([time_ip])
        value_ip = np.linspace(float(nthreads_processes_int[1][0]), float(nthreads_processes_int[1][-1]), 100)
        nthreads_processes_int_y = np.array([value_ip])
        
        
        axes[i].plot((nthreads_processes_int_x[0]), nthreads_processes_int_y[0])
        axes[i].legend(['Measurement', 'IP Measurement'])
        axes[i].set_title('Nthreads')
        axes[i].set_ylabel('amount')
         # ToDo add units to axes
    
fig.suptitle('nthreads', fontsize=16)
# plt.show()


fig, axes= plt.subplots(occurrences, 1, figsize=(15, 40))
fig.text(0.5, 0.04, 'time', ha='center', va='center')
fig.text(0.03, 0.5, 'Performance', ha='center', va='center', rotation='vertical')

# Plots of the Memory usage behaviour over time for all containers
for i in range(0,occurrences-1):
    mem_processes_int = memory[:,i:length-1:occurrences]
    for j in range(0,len(cpu_processes)-2):
        tck = interpolate.splrep((mem_processes_int[0]), (mem_processes_int[j+1]), s=0)
        ip = np.array([interpolate.splev(bm_ip[0], tck, der=0)])
        time_ip = np.linspace(float(mem_processes_int[0][0]), float(mem_processes_int[0][-1]), 100)
        mem_processes_int_x = np.array([time_ip])
        value_ip = np.linspace(float(mem_processes_int[1][0]), float(mem_processes_int[1][-1]), 100)
        mem_processes_int_y = np.array([value_ip])
        
        
        axes[i].plot((mem_processes_int_x[0]), mem_processes_int_y[0])
        axes[i].legend(['Measurement', 'IP Measurement'])
        axes[i].set_title('mem')
        axes[i].set_ylabel('%')
        # ToDo add units to axes
    
fig.suptitle('mem', fontsize=16)